In [2]:
from langchain_openai import AzureChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END, Graph 
from models.database import *
from models.models import *
import langgraph
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
MODELO = os.getenv("MODELO")
KEY = os.getenv("KEY")
VERSION = os.getenv("VERSION")
ENDPOINT = os.getenv("ENDPOINT")

In [4]:
llm = AzureChatOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT,
    azure_deployment = MODELO
    )

In [ ]:
import openai
openai_client = openai.AzureOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT
    )

In [ ]:
messages=[{"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": "¿Cuál es la capital de España?"}]

response = openai_client.chat.completions.create(
    model = MODELO,
    messages = messages,
    max_tokens=100
    )
response.choices[0].message.content

    ("pendiente_condicionada", "flujo"),
    ("pendiente_no_condicionada", "flujo"),
    ("grabar_cesta_srm", "flujo"),
    ("solicitar_firma_inf_prov_cond", "flujo"),
    ("firma_inf_prov_cond", "flujo"),
    ("lanzar_cesta_srm", "flujo"),
    ("aprobar_compra_manager", "aprobacion"),
    ("aprobar_compra_gestion", "aprobacion"),
    ("revision_compra", "flujo"),
    ("solicitar_oferta_provedoores", "flujo"),
    ("proveedor_envia_ofertn", "flujo"),
    ("consulta_acep_oferta", "flujo"),
    ("negociacion_proveedores", "flujo"),
    ("proponer_proveedor", "flujo"),
    ("aprobar_adjuducacion", "aprobacion"),
    ("facturar", "flujo"),
    ("fin", "flujo"),

In [ ]:
# Agrego un proveedor de prueba
from sqlalchemy.orm import Session
from models.database import engine
from models.models import Proveedor

def agregar_proveedor_prueba():
    """
    Agrega un proveedor de prueba a la base de datos.
    """
    session = Session(bind=engine)

    # Crear un proveedor de prueba
    nuevo_proveedor = Proveedor(
        nombre="Apple",
        contacto="Soporte Apple",
        email="apple@proveedor.com",
        estado="aprobado"
    )
    session.add(nuevo_proveedor)
    session.commit()

    print(f"✅ Proveedor de prueba añadido: {nuevo_proveedor.nombre} (ID: {nuevo_proveedor.id_proveedor})")
    session.close()

# Ejecutar la función para agregar el proveedor
agregar_proveedor_prueba()


In [22]:
def obtener_datos_pedido(entrada_usuario):
    ''' Nodo que usa IA para obtener los datos del pedido del usuario, crea la cesta y el pedido con el estado inicial,
      y decide si el flujo valida el proveedor o graba la cesta.
    '''
    # Se crea la sesión de SQLAlchemy	
    session = Session(bind=engine)

    prompt = f''' 
    Eres el encargado de obtener la información de un pedido.
    Se necesita conocer:
    - Nombre del proveedor (si no se menciona, deja "desconocido")
    - Moneda en la que se realizó la compra
    - Presupuesto estimado
    - Producto a comprar

    {entrada_usuario}

    Devuelve los datos en un formato claro como:
    "Proveedor: Proveedor X, Moneda: EUR, Presupuesto: 5000, Producto: [Producto Y]"
    '''
    response = llm.invoke(prompt)
    datos_ia = response.content.strip()

    print(f'IA: {datos_ia}')

    # Extraer los datos del texto
    datos_extraidos = {}
    for item in datos_ia.split(','):
        key, value = item.strip().split(':')
        datos_extraidos[key.lower().strip()] = value.strip()
    
    nombre_proveedor = datos_extraidos.get('proveedor', 'desconocido')
    codigo_moneda = datos_extraidos.get('moneda', 'EUR') # Por defecto EUR
    presupuesto = float(datos_extraidos.get('presupuesto', 1000)) # Por defecto 1000

    print(f"📦 Datos extraídos: Proveedor: {nombre_proveedor}, Moneda: {codigo_moneda}, Presupuesto: {presupuesto}")
    
    # Verificar si el proveedor ya existe en la BD
    proveedor = session.query(Proveedor).filter_by(nombre=nombre_proveedor).first()
    proveedor_id = proveedor.id_proveedor if proveedor else None

    if proveedor:
        print(f"✅ Proveedor '{proveedor.nombre}' encontrado en la base de datos.")
        es_nuevo = False
        estado_inicial_nombre = "pendiente_no_condicionada"
    else:
        print(f"⚠️ Proveedor '{nombre_proveedor}' no encontrado. Requiere validación.")
        es_nuevo = True
        estado_inicial_nombre = "pendiente_condicionada"
    
    # Obtener estado inicial correspondiente
    estado_inicial = session.query(Estado).filter_by(nombre=estado_inicial_nombre).first()
    if not estado_inicial:
        print(f"⚠️ Estado '{estado_inicial_nombre}' no encontrado en la base de datos.")
        return {"error": "Estado inicial no encontrado"}



    # Verificar que la moneda existe en la BD
    moneda = session.query(Moneda).filter(Moneda.codigo == codigo_moneda).first()
    moneda_id = moneda.id_moneda if moneda else None

    if not moneda:
        print(f"⚠️ Moneda '{codigo_moneda}' no encontrada. Se usará EUR por defecto.")
        moneda = session.query(Moneda).filter_by(codigo="EUR").first()
        moneda_id = moneda.id_moneda

    # Crear la cesta y el pedido
    nueva_cesta = Cesta(
        nombre="Cesta generada automáticamente",
        descripcion="Pedido generado automáticamente",
        tipo_compra="ordinaria",
        presupuesto=presupuesto,
        usuario_sap_id=1,
        proveedor_id=proveedor_id,
        moneda_id=moneda_id,
        contrato_id=None
    )
    session.add(nueva_cesta)
    session.commit()

    print(f"✅ Cesta creada con ID: {nueva_cesta.id_cesta}")

    nuevo_pedido = Pedido(
        posicion=1,
        tipo="Ordinario",
        pedido_tipoimp="K",
        descripcion=f"Compra de {datos_extraidos.get('productos', 'productos no especificados')}",
        id_proveedor=proveedor_id,
        cesta_id=nueva_cesta.id_cesta,  # La Cesta ya está asignada
        estado_tramitacion_id=estado_inicial.id_estado,
        creador_id=1,
        moneda_id=moneda_id
    )
    session.add(nuevo_pedido)
    session.commit()

    # Se extraen las variables antes de cerrar la sesion 

    pedido_id = nuevo_pedido.id_pedido
    cesta_id = nueva_cesta.id_cesta
    session.close()

    return {

        "id_pedido": pedido_id,
        "id_proveedor": proveedor_id,  # ✅ Ya no intenta acceder a un objeto desvinculado
        "nombre_proveedor": nombre_proveedor,
        "id_cesta": cesta_id,
        "id_moneda": moneda_id,  # ✅ Ya no intenta acceder a un objeto desvinculado
        "estado_actual": estado_inicial_nombre,
        "id_usuario": 1,
        "nombre_cesta": "Cesta generada automáticamente",
        "tipo_compra": "ordinaria",
        "presupuesto": presupuesto,
        "descripcion_pedido": "Pedido generado automáticamente",
        "pedido_tipoimp": "K",
        "es_nuevo": es_nuevo  # 🔹 Indica si el proveedor debe validarse antes de continuar
    }


In [23]:
consulta_usuario = """
Necesito comprar 15 monitores y 10 teclados mecánicos para la oficina. 
El presupuesto estimado es de 7000 dólares. 
Quiero hacer la compra a Apple, pero si no es posible, podemos buscar otro proveedor.
"""

# Probar obtener_datos_pedido con una consulta de usuario
resultado = obtener_datos_pedido(consulta_usuario)

# Mostrar los resultados obtenidos
print("\n🔹 RESULTADO DE LA FUNCIÓN 🔹")
for clave, valor in resultado.items():
    print(f"{clave}: {valor}")

# Verificar si el proveedor fue detectado correctamente
if resultado["es_nuevo"]:
    print("\n⚠️ El proveedor es nuevo y requiere validación.")
else:
    print("\n✅ El proveedor ya existe en la base de datos y se usará directamente.")


IA: Proveedor: Apple, Moneda: USD, Presupuesto: 7000, Producto: 15 monitores y 10 teclados mecánicos
📦 Datos extraídos: Proveedor: Apple, Moneda: USD, Presupuesto: 7000.0
2025-03-17 23:17:36,525 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-17 23:17:36,526 INFO sqlalchemy.engine.Engine SELECT proveedores.id_proveedor AS proveedores_id_proveedor, proveedores.nombre AS proveedores_nombre, proveedores.contacto AS proveedores_contacto, proveedores.email AS proveedores_email, proveedores.estado AS proveedores_estado, proveedores.fecha_registro AS proveedores_fecha_registro 
FROM proveedores 
WHERE proveedores.nombre = ?
 LIMIT ? OFFSET ?
2025-03-17 23:17:36,526 INFO sqlalchemy.engine.Engine [cached since 1.59e+04s ago] ('Apple', 1, 0)
✅ Proveedor 'Apple' encontrado en la base de datos.
2025-03-17 23:17:36,528 INFO sqlalchemy.engine.Engine SELECT estados.id_estado AS estados_id_estado, estados.nombre AS estados_nombre, estados.tipo AS estados_tipo, estados.requiere_aprobacion AS est

In [ ]:
def grabar_cesta_srm():
    tools = []
    agent_pedido = create_react_agent(llm, tools)


In [ ]:
from sqlalchemy.orm import Session
from models.database import engine
from models.models import Cesta, Pedido, Estado, Transicion

def grabar_cesta_srm(datos):
    """
    Nodo que crea una Cesta y un Pedido en la base de datos,
    y actualiza el estado del Pedido basado en la lógica de transiciones.
    """
    session = Session(bind=engine)

    # 📌 Extraer datos obtenidos
    id_proveedor = datos["id_proveedor"]
    id_usuario = datos["id_usuario"]
    nombre_cesta = datos["nombre_cesta"]
    tipo_compra = datos["tipo_compra"]
    presupuesto = datos["presupuesto"]
    id_moneda = datos["id_moneda"]
    descripcion_pedido = datos["descripcion_pedido"]
    pedido_tipoimp = datos["pedido_tipoimp"]

    # 📌 Obtener el estado inicial del pedido
    estado_inicial_nombre = "pendiente_condicionada" if datos["es_nuevo"] else "pendiente_no_condicionada"
    estado_inicial = session.query(Estado).filter_by(nombre=estado_inicial_nombre).first()

    if not estado_inicial:
        print("❌ Error: No se encontró el estado inicial.")
        return {"error": "Estado inicial no encontrado"}

    # 📌 Crear la Cesta en la BD
    nueva_cesta = Cesta(
        nombre=nombre_cesta,
        tipo_compra=tipo_compra,
        presupuesto=presupuesto,
        usuario_sap_id=id_usuario,
        proveedor_id=id_proveedor,
        moneda_id=id_moneda
    )
    session.add(nueva_cesta)
    session.commit()

    print(f"✅ Cesta creada con ID: {nueva_cesta.id_cesta}")

    # 📌 Crear el Pedido basado en la Cesta
    nuevo_pedido = Pedido(
        posicion=1,
        tipo="Ordinario",
        pedido_tipoimp=pedido_tipoimp,
        descripcion=descripcion_pedido,
        id_proveedor=nueva_cesta.proveedor_id,
        cesta_id=nueva_cesta.id_cesta,
        estado_tramitacion_id=estado_inicial.id_estado,  # Estado inicial
        creador_id=id_usuario,
        moneda_id=nueva_cesta.moneda_id
    )
    session.add(nuevo_pedido)
    session.commit()

    print(f"✅ Pedido {nuevo_pedido.id_pedido} creado en estado '{estado_inicial.nombre}'")

    # 📌 Buscar la transición correcta para actualizar el estado
    transicion = session.query(Transicion).filter_by(
        estado_origen_id=estado_inicial.id_estado
    ).first()

    if transicion:
        nuevo_estado = session.query(Estado).filter_by(id_estado=transicion.estado_destino_id).first()
        if nuevo_estado:
            nuevo_pedido.estado_tramitacion_id = nuevo_estado.id_estado
            session.commit()
            print(f"🔄 Pedido {nuevo_pedido.id_pedido} actualizado a estado '{nuevo_estado.nombre}'")
        else:
            print("❌ No se encontró el estado de destino.")
    else:
        print("⚠️ No se encontró una transición válida para este estado.")

    session.close()

    return {"id_pedido": nuevo_pedido.id_pedido, "estado_actual": nuevo_pedido.estado_tramitacion_id}


In [ ]:
def aprobacion_proveedor():
    tools = []
    agent_pedido = create_react_agent(llm, tools)

In [ ]:
# Codigo para generar el flujo

graph_builder = StateGraph(dict)
graph_builder = Graph()

# Nodos al grafo

graph_builder.add_node("obtener_datos_del_pedido", obtener_datos_pedido)
graph_builder.add_node("grabar_cesta_srm", grabar_cesta_srm)
graph_builder.add_node("aprobacion_proveedor", aprobacion_proveedor)

In [ ]:
# 📌 Conectar nodos condicionalmente
def decidir_siguiente_nodo(datos):
    if datos["es_nuevo"]:
        return "aprobacion_proveedor"
    return "crear_pedido"

In [ ]:
graph.add_conditional_edges("obtener_datos_pedido", decidir_siguiente_nodo)
graph.add_edge("aprobacion_proveedor", "crear_pedido")  # Si se aprueba, se crea el pedido
